# More pandas!

In [1]:
import os
import pandas as pd
import io
import numpy as np


## Reading data from URLs

In [2]:
url1 = "https://raw.githubusercontent.com/nyusterndatabootcamp"

url2 = "/data_resources/master/test.csv"

url  = url1 + url2        

df = pd.read_csv(url)  

In [3]:
url

'https://raw.githubusercontent.com/nyusterndatabootcamp/data_resources/master/test.csv'

In [4]:
df

,name,x1,x2,x3
0,Dave,1,2,3.5
1,Chase,4,3,4.3
2,Spencer,5,6,7.8


In [5]:
df_only2 = pd.read_csv(url, nrows = 2)

df_only2

,name,x1,x2,x3
0,Dave,1,2,3.5
1,Chase,4,3,4.3


In [ ]:
df_notone = pd.read_csv(url, na_values = 1)

df_notone

In [ ]:
url3 = "/data_resources/master/test.xls"
url = url1 + url3

df_excel = pd.read_excel(url, na_values = 1) # Simmilar funcitonality!

df_excel

## What about Google sheets?

In [ ]:
import gspread as gs

In [ ]:
# You will need to generate a project, a credential that is a key (saved to your computer)
# Once you create the key, you need to go IAM and admin go to service accounts
# Share your spreadsheet with the e-mail address there

In [ ]:
gc = gs.service_account(filename='my-project-for-class-326120-d3300c932c44.json')


In [ ]:
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Rj3q1uezUg0tbKkmaczk86vgFhSoa17j3rmXzaLXTyI/edit?usp=sharing')


In [ ]:
sh

In [ ]:
ws = sh.worksheet('Bakery')


In [ ]:
ws

In [ ]:
df_gs = pd.DataFrame(ws.get_all_records())
df_gs.head()

## Reading from your computer

In [ ]:
my_path = os.getcwd()
my_path

In [ ]:
df.to_csv(my_path + '/'+'df1.csv')

df.to_excel(my_path + '/'+'df1.xlsx')



In [ ]:
df_new = pd.read_excel("df1.xlsx")

In [ ]:
df_new

In [ ]:
df_new = pd.read_excel(my_path + "/" + "df1.xlsx")


In [ ]:
df_new

## Let's work with some actual data now (Chipotle)

![Hero we don't deserve](https://pics.me.me/chipotle-why-is-guac-extra-people-over-27248804.png)

In [ ]:
chipurl=url = 'https://raw.githubusercontent.com/TheUpshot/chipotle/master/orders.tsv'

df_chipotle=pd.read_csv(chipurl, sep='\t')

In [ ]:
df_chipotle

In [ ]:
df_chipotle.describe()

In [ ]:
df_chipotle.dtypes

In [ ]:
#See head and tail

df_chipotle.head(5)

In [ ]:
df_chipotle.tail(6)

In [ ]:
df_chipotle.item_price[0]

In [ ]:
#HOW DO WE SOLVE THIS SO WE CAN DEAL WITH DOLLARS AS A FLOAT?



In [ ]:
float(df_chipotle.item_price[0].replace("$","").replace(" ",""))

In [ ]:
float(df_chipotle.item_price.replace("$","").replace(" ",""))

In [ ]:
df_chipotle.item_price.replace("$","").replace(" ","")

In [ ]:
len(df_chipotle.item_price)

In [ ]:
#Try making it with a list
floatpricelist=[float(df_chipotle.item_price[i].replace("$","").replace(" ",""))
 for i in range(0, len(df_chipotle['item_price']))]

In [ ]:
# We can just add them as a column

df_chipotle['floatprice']=floatpricelist
df_chipotle.head()

In [ ]:
#Let's drop it

df_chipotle.drop(columns=['floatprice'], inplace=True)
df_chipotle.head()

In [ ]:
# This all feels too convoluted... what about writing a function?

def dollartofloat(x):
    flprice=float(x.replace("$","").replace(" ",""))
    return flprice

In [ ]:
#There is a way to use this function, pandas lets us use a method called apply


df_chipotle['floatprice']=df_chipotle['item_price'].apply(dollartofloat)
df_chipotle.head()

In [ ]:
#Let's drop it and make it in a single line
df_chipotle.drop(columns=['floatprice'], inplace=True)


In [ ]:
df_chipotle['floatprice']=df_chipotle['item_price'].apply(lambda x: float(x.replace("$","").replace(" ","")) )
df_chipotle.head()

This is all good and nice, but I want to start using this table for something useful...For example, I'd like to know how much each person paid for each order and have it in a dataframe

## Grouping

In [ ]:
# First let's get the revenue per item

df_chipotle['rev_item']=df_chipotle['quantity']*df_chipotle['floatprice']
df_chipotle.head()


We can identify each order by order_id! So all we need to have is the sum of revenue per item!

In [ ]:
df_chipotle.groupby('order_id').sum()

In [ ]:
df_chipotle.groupby('order_id')['rev_item'].sum()

In [ ]:
df_order=df_chipotle.groupby('order_id')['rev_item'].sum().reset_index()

df_order.head()

In [ ]:
df_order.rename(columns={'rev_item':'rev_order'}, inplace=True)


In [ ]:
df_order.head()

Can we do better? Can we create a new dataframe that on top of all of this gives us not only the revenue for the order, but the average revenue per item and the max revenue per item for each order? 

In [ ]:
df_chipotle.groupby('order_id')['rev_item'].agg(rev_order='sum', 
                                               ave_item_rev=np.mean,
                                               max_item_rev=np.max)
                           

In [ ]:
df_order_detail=df_chipotle.groupby('order_id')['rev_item'].agg(rev_order='sum', 
                                               ave_item_rev=np.mean,
                                               max_item_rev=np.max).reset_index()
                           

In [ ]:
df_order_detail

WOOPS! I wanted to get the standard deviation too but I forgot. Let me make a new goupby for it

In [ ]:
df_std_chipotle=df_chipotle.groupby('order_id')['rev_item'].agg(
    std_item_rev=np.std).reset_index() 
df_std_chipotle.head()                                               

In [ ]:
#Why a NaN
df_chipotle[df_chipotle.order_id==2]

In [ ]:
# we will deal with this later
#For now let's join tables together

df_merged=df_order_detail.merge(df_std_chipotle, how='inner', on='order_id')
df_merged.head()

In [ ]:
#why not add it to our large dataframe making it the left!

df_large_left=df_chipotle.merge(df_merged, how='left', on='order_id')
df_large_left.head()